In [8]:
#By Arpan Mitra
import pandas as pd
import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Load data
movies_data =pd.read_csv("C:\\Users\\KIIT\\Downloads\\PRJ Movie Recommendation System-20240710T145226Z-001\\PRJ Movie Recommendation System\\movies.csv")

df = movies_data.copy()
display(df.head())

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [10]:
#displaying null values
print(df.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64


In [11]:
# Data preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    return ' '.join(words)

selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

for feature in selected_features:
    df[feature] = df[feature].fillna('')
    df[feature] = df[feature].apply(preprocess_text)

# Fill missing values using mode imputation
imputer = SimpleImputer(strategy='most_frequent')
df[selected_features] = imputer.fit_transform(df[selected_features])
print(df.isna().sum())

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64


In [12]:
# Combine features
combined_features = df['genres'] + ' ' + df['keywords'] + ' ' + df['tagline'] + ' ' + df['cast'] + ' ' + df['director']

# Weighted TF-IDF
tfidf = TfidfVectorizer(sublinear_tf=True, use_idf=True)
feature_vectors = tfidf.fit_transform(combined_features)
display (feature_vectors.shape)
print (feature_vectors)

# Content-based similarity
similarity = cosine_similarity(feature_vectors)
print(similarity)

(4803, 15648)

  (0, 196)	0.08123373023729306
  (0, 254)	0.0932345684729635
  (0, 4780)	0.1146507115923348
  (0, 12500)	0.10711391112954896
  (0, 4924)	0.10649460688132552
  (0, 3357)	0.2192744724243351
  (0, 2823)	0.22952471072343467
  (0, 5284)	0.17019254091305186
  (0, 13216)	0.2971530583329036
  (0, 14970)	0.1295767415055915
  (0, 2968)	0.25279043015199404
  (0, 13113)	0.22108925698327955
  (0, 4493)	0.2483084038192933
  (0, 15323)	0.13133055358992712
  (0, 10611)	0.28123026925840516
  (0, 12272)	0.15524552610505582
  (0, 15334)	0.24435497641183895
  (0, 15618)	0.20874645043184406
  (0, 12246)	0.22504268439073394
  (0, 12927)	0.21286163881764375
  (0, 15037)	0.20508113928305965
  (0, 13421)	0.15472512284290532
  (0, 8014)	0.20874645043184406
  (0, 9370)	0.15524552610505582
  (0, 11967)	0.2007441038369028
  :	:
  (4801, 365)	0.18459550404394684
  (4801, 3208)	0.27566435020825936
  (4801, 4394)	0.2573418693517928
  (4801, 15595)	0.3005264075437804
  (4801, 12714)	0.2902077214403185
  (4801, 12113)	

In [13]:
def recommend_movies(movie_name):
    list_of_all_titles = df['title'].tolist()
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    close_match = find_close_match[0]
    index_of_the_movie = df[df.title == close_match]['index'].values[0]

    similarity_score = list(enumerate(similarity[index_of_the_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    print('Movies suggested for you : \n')
    i = 1
    for movie in sorted_similar_movies:
        index = movie[0]
        title_from_index = df[df.index == index]['title'].values[0]
        if i < 30:
            print(i, '.', title_from_index)
            i += 1

In [14]:
# Example usage
movie_name = input('Enter your favourite movie name : ')
recommend_movies(movie_name)

Enter your favourite movie name : Arabian Night
Movies suggested for you : 

1 . Fright Night
2 . House of D
3 . Marmaduke
4 . Anderson's Cross
5 . 30 Days of Night
6 . Action Jackson
7 . The Girl with the Dragon Tattoo
8 . Kick-Ass 2
9 . Broken Horses
10 . The Beaver
11 . Cabin Fever
12 . Green Room
13 . Dying of the Light
14 . The Imaginarium of Doctor Parnassus
15 . Hearts in Atlantis
16 . Charlie Bartlett
17 . Terminator Salvation
18 . Seven Psychopaths
19 . Rudderless
20 . Let Me In
21 . Role Models
22 . The Last Shot
23 . Dracula 2000
24 . Drive Me Crazy
25 . The New World
26 . Million Dollar Arm
27 . Krampus
28 . The Pallbearer
29 . Superbad
